In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

c:\Users\Daksh Vashist\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
# [1] is used to select the specific table needed from the Wikipedia website
asx_data = pd.read_html('https://en.wikipedia.org/wiki/S%26P/ASX_200')[1]

In [4]:
asx_data.head(2)

,Code,Company,Sector,Market Capitalisation,Chairperson,HQ
0,A2M,a2 Milk Company,Consumer Staples,4.222573e+09,David Hearn,Auckland
1,ABC,Adbri,Materials,2.114513e+09,Raymond Barro,Adelaide


In [5]:
# Put tickers into a list
tickers = asx_data['Code'].to_list()

In [6]:
# Only need the ['Adj close'] using yfinance
prices = yf.download(tickers, start='2008-01-01', end='2023-05-31')['Adj Close']

[*********************100%%**********************]  200 of 200 completed

39 Failed downloads:
['SGM', 'DEG', 'BKW', 'CCP', 'MPL', 'IRE', 'HLS', 'ABP', 'COH', 'STO', 'TAH', 'CHC', 'RWC', 'CRN', 'ANN', 'TNE']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2008-01-01 -> 2023-05-31)')
['HDN', 'BKL', 'JBH', 'UWL', 'VUK']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['SQ2', 'CXO', 'CCX', 'BRG', 'GNC', 'S32', 'APE', 'A2M', 'IVC', 'ABC', 'SFR', 'AKE', 'PME', 'MP1', 'TCL', 'NHF', 'WBC', 'TLC']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [7]:
# Change to datetime
prices.index = pd.to_datetime(prices.index)

In [8]:
# Convert from daily to monthly prices using 'M'
prices_monthly = prices.resample('M').last()

In [9]:
# Drop the columns with insufficient data
prices_monthly_clean = prices_monthly.dropna(axis=1)

In [10]:
prices_monthly_clean.head()

Ticker,AIA,ALL,ALX,AMP,APA,ASX,AUB,BAP,BEN,BHP,...,PDN,PNI,PPT,PRU,RIO,RMD,SOL,VEA,WDS,WOR
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-31,31.082815,33.082664,128.790054,38.967983,74.934425,2.147642,12.353294,55.942844,20.415468,32.559658,...,15.137025,5.906594,1.749220,47.862827,36.329906,19.654076,62.549999,27.475292,20.361635,5.334654
2008-02-29,31.011572,32.081169,111.261200,35.677792,90.222687,2.323679,11.304968,57.978706,18.509068,35.542202,...,15.652360,5.253389,1.714139,41.543354,40.870754,17.132269,53.150002,27.196693,25.424765,5.707818
2008-03-31,30.783543,32.578342,130.408615,36.530270,95.028427,2.429301,12.520137,55.958454,19.059607,31.982155,...,15.585384,5.617073,1.703275,44.549377,37.038651,17.847342,56.799999,27.299555,24.226864,5.566880
2008-04-30,34.232430,34.137424,131.070801,33.565243,106.044090,2.595277,12.384402,63.921139,18.698025,39.175106,...,16.135851,5.726695,1.756653,43.103306,42.269245,18.245083,83.750000,28.767330,25.620340,5.943063
2008-05-31,32.992523,34.813919,126.913887,33.402668,105.555931,2.640544,13.089655,66.387001,19.890846,40.962418,...,16.521194,5.736608,1.784575,42.528301,43.438408,16.666834,125.400002,29.114145,30.656393,6.579937


In [11]:
# Use "shift() to move the dataframe up and down relate to the index
momentum_12 = (prices_monthly_clean/prices_monthly_clean.shift(12))-1
monthly_returns = (prices_monthly_clean/prices_monthly_clean.shift(1))-1

In [12]:
# Drop the 1 or 2 rows with NA's
momentum_12 = momentum_12.dropna(axis=0)
monthly_returns = monthly_returns.dropna(axis=0)

In [13]:
# Establish a df for the ranks
quintile_ranks = pd.DataFrame(index=momentum_12.index, columns=momentum_12.columns)

In [14]:
# Place each stock in a "quintile" bucket, at each "date" (rows)
for date in momentum_12.index:
    row_values = momentum_12.loc[date] # looping values each row
    ranks = pd.Series(row_values).rank(method='max') # set rank at each row
    quintiles = pd.qcut(ranks, q=5, labels=False) # divide into quintiles

    # Create dataframe with ranks for each stock at each month(4=high, 0=low)
    quintile_ranks.loc[date] = quintiles

In [15]:
quintile_ranks.head(5)

Ticker,AIA,ALL,ALX,AMP,APA,ASX,AUB,BAP,BEN,BHP,...,PDN,PNI,PPT,PRU,RIO,RMD,SOL,VEA,WDS,WOR
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-31,1,1,1,0,3,0,3,2,1,2,...,2,2,3,0,0,4,0,1,1,2
2009-02-28,2,0,1,0,2,1,3,2,1,2,...,2,3,3,0,0,4,0,1,1,1
2009-03-31,2,0,1,0,1,2,3,2,1,3,...,1,2,3,0,0,4,0,1,1,2
2009-04-30,2,0,1,1,1,2,4,2,2,1,...,2,3,3,0,0,3,0,1,1,4
2009-05-31,2,0,3,1,1,1,2,2,1,2,...,2,3,3,0,0,4,0,1,1,3


In [16]:
quintile_dfs = {} 
portfolio_returns = pd.DataFrame()

# Iterate for each quintile, and form portfolios accordingly
for quintile in range(5):
    
    # Only take returns if they're in quintile associated with the current loop.
    filtered_df = monthly_returns[quintile_ranks == quintile]
    
    # shift to "t+1" return as rank at time "t" corresponds to return at time "t+1"
    filtered_df_shifted = filtered_df.shift(-1).dropna(axis=0)

    # Put those returns into dictionary "quintile_dfs"
    quintile_dfs[quintile] = filtered_df_shifted

    #For Equal-Weight, simply take the average return across stocks within each quintile dataframe
    portfolio_returns[quintile] = quintile_dfs[quintile].mean(axis=1).dropna()


# Plot the cumulative return
(1+portfolio_returns).cumprod().hvplot(title = 'Portfolios - Log Scale', height = 300, width= 500, grid=True, logy=True)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [17]:
# Calculate Ann Return
cumulative_return = (1 + portfolio_returns).prod() - 1
num_periods = len(monthly_returns)
annualized_return = (1 + cumulative_return) ** (12 / num_periods) - 1

# Calculate Ann Vol
ann_vol = portfolio_returns.std()*math.sqrt(12)

# Calculate Risk-adjusted-return
risk_adj_return = annualized_return/ann_vol
risk_adj_return.hvplot(kind='bar',title = 'Quintile Momentum portfolio - Risk Adjusted Return', height = 300, width= 500, grid=True, color = 'teal')

:Bars   [index]   (0)